<a href="https://colab.research.google.com/github/YoshiyukiKono/gen_ai-sandbox/blob/main/vertex_ai_dry_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://cloud.google.com/blog/ja/products/ai-machine-learning/generative-ai-applications-with-vertex-ai-palm-2-models-and-langchain

In [1]:
!pip install langchain

In [3]:
from langchain.chains import RetrievalQA
from langchain.llms import VertexAI

In [2]:
!pip install google-cloud-aiplatform>=1.35.0

https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm


Google Cloud does not use customer data to train its foundation models as part of Google Cloud's AI/ML Privacy Commitment. More details about how Google processes data can also be found in Google's Customer Data Processing Addendum (CDPA).

To use Vertex AI PaLM you must have the google-cloud-aiplatform Python package installed and either:

 - Have credentials configured for your environment (gcloud, workload identity, etc...)
 - Store the path to a service account JSON file as the GOOGLE_APPLICATION_CREDENTIALS environment variable
This codebase uses the google.auth library which first looks for the application credentials variable mentioned above, and then looks for system-level auth.

For more information, see:

https://cloud.google.com/docs/authentication/application-default-credentials#GAC
https://googleapis.dev/python/google-auth/latest/reference/google.auth.html#module-google.auth

In [9]:
!ls -1

'=1.35.0'
genai-399700-51747f1848fb.json
sample_data


In [25]:
%env GOOGLE_APPLICATION_CREDENTIALS=./genai-399700-51747f1848fb.json

env: GOOGLE_APPLICATION_CREDENTIALS=./genai-399700-51747f1848fb.json


In [33]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

./genai-399700-51747f1848fbXXXXXXX.json


In [28]:
!ls -l $GOOGLE_APPLICATION_CREDENTIALS

-rw-r--r-- 1 root root 2350 Nov  2 01:07 ./genai-399700-51747f1848fb.json


In [40]:
llm = VertexAI()

In [41]:
query="Generative AIとは？"

In [46]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

#prompt_template = "What is a good name for a company that makes {product}?"
prompt_template = "{query}?"

llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain(query)

{'query': 'Generative AIとは？',
 'text': ' Generative AIとは、テキスト、コード、画像、音声、動画など、新しいコンテンツを生成することができる人工知能（AI）の一種です。Generative AIは、機械学習アルゴリズムを使用して、既存のデータから新しいコンテンツを生成します。Generative AIは、様々な分野で使用されており、新しい製品やサービスの開発、コンテンツの作成、研究の支援など、様々な用途に使用されています。'}

In [42]:
# QA チェーンを作成してソース ドキュメントとともにユーザーのクエリに回答する
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    #retriever=retriever,
    return_source_documents=True)


# QA チェーンを実行する
result = qa({"query": query})

ValidationError: ignored

In [8]:
from google.cloud import aiplatform

aiplatform.init(
    project=project,
    location=location,
    experiment=experiment,
    staging_bucket=staging_bucket,
    credentials=credentials,
    encryption_spec_key_name=encryption_spec_key_name,
    service_account=service_account,
)

NameError: ignored

In [47]:
# LLM を定義して回答を生成する
llm = VertexAI(model_name='text-bison@001', max_output_tokens=512, temperature=0.2)

https://stackoverflow.com/questions/52777183/attributeerror-str-object-has-no-attribute-before-request

In [37]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("./genai-399700-51747f1848fb.json")

In [38]:
from google.cloud import aiplatform

project = "genai-399700"
#location = "us-west1"
aiplatform.init(project=project, credentials=credentials)

In [39]:
# LLM を定義して回答を生成する
llm = VertexAI(model_name='text-bison@001', max_output_tokens=512, temperature=0.2)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import VertexAI

# Matching Engine を Vector Store として定義する
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=ME_REGION,
    gcs_bucket_name=f'gs://{ME_BUCKET_NAME}',
    embedding=embedding,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID
)


# Matching Engine のインデックスを Retriever インターフェースとして公開する
retriever = me.as_retriever(
    search_type="similarity",
    search_kwargs={"k":NUMBER_OF_RESULTS})


# LLM を定義して回答を生成する
llm = VertexAI(model_name='text-bison@001', max_output_tokens=512, temperature=0.2)

# QA チェーンを作成してソース ドキュメントとともにユーザーのクエリに回答する
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)


# QA チェーンを実行する
result = qa({"query": query})